In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import AIMessage, HumanMessage
from langchain.schema.runnable import RunnablePassthrough

# ✅ 1️⃣ LLM 모델 설정
llm = ChatOpenAI(temperature=0.3)

# ✅ 2️⃣ 메모리 설정 (대화 기록 저장)
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

# ✅ 3️⃣ Few-Shot 예제 설정 (영화 이모티콘 변환 예제)
examples = [
    {"title": "탑건", "emoji": "✈️🔥🎶"},
    {"title": "대부", "emoji": "🎩🔫🍝"},
    {"title": "덤앤더머", "emoji": "🤣🚙🧥"},
    {"title": "나홀로 집에", "emoji": "🏠🎄😂"},
]

few_shot_prompt = FewShotChatMessagePromptTemplate.example_prompt(
    examples,
    ChatPromptTemplate.from_messages([
        ("system", "당신은 영화 제목을 3개의 이모티콘으로 변환하는 AI입니다."),
        ("human", "{title}"),
    ]),
    example_key="title",
    output_key="emoji"
)

# ✅ 4️⃣ 메모리를 불러오는 함수
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

# ✅ 5️⃣ LCEL 체인 구성
chain = RunnablePassthrough.assign(chat_history=load_memory) | few_shot_prompt | llm

# ✅ 6️⃣ 체인 실행 함수
def get_movie_emoji(title):
    response = chain.invoke({"title": title})
    memory.save_context({"input": title}, {"output": response.content})
    print(f"{title} -> {response.content}")

# ✅ 7️⃣ 실행 테스트 (두 개의 영화 질문 후, 이전 질문 확인)
get_movie_emoji("인셉션")
get_movie_emoji("타이타닉")

# ✅ 8️⃣ 이전 질문한 영화가 무엇인지 묻기
response = chain.invoke({"title": "내가 이전에 질문한 영화 제목은?"})
print(response.content)

AttributeError: type object 'FewShotChatMessagePromptTemplate' has no attribute 'from_examples'